# 公式拟合

某医院进行临床登记实验, 收集白内障病例2000眼, 进行Phaco+IOL植入术, 使用SRK-2公式进行IOL计算, 

术前检查记录: 
* 选用IOL的A常数: A
* 角膜曲率: K1, K2
* 眼轴长: L
* 目标屈光度: REF

术中记录: 
* 植入IOL度数: P

术后3个月复查: 
* 仅记录了术后对屈光状态不满意, 给予眼镜处方患者的显然验光结果: PostREF
* 一部分病人失访, 术后验光结果空缺

现该医院委托你拟合一个新的IOL计算公式, 所有的病例数据已经整理好存成excel文件, 放在了data文件夹内的sampleIOLdataset.xlsx文件中. 


# 读取数据集

In [14]:
import os
import pandas as pd
pathname='data'
fname='sampleIOLdataset.xlsx'
filename=os.path.join(pathname,fname)
IOLdata=testdata=pd.read_excel(filename)

In [15]:
print(IOLdata[0:5])
print(IOLdata.A[0:3])


       A         K1         K2          L   REF   IOLPower  PostOPREF
0  118.8  41.351356  42.312151  24.171811 -0.50  21.346894  -0.250431
1  118.4  43.621478  43.501080  24.586358 -3.25  21.291454  -1.088519
2  118.4  42.109335  38.567343  24.283508 -3.25  25.449224  -0.981862
3  118.8  37.217467  41.210893  23.503847 -3.25  28.810120  -1.493851
4  119.0  42.110941  38.590989  23.361435 -3.25  28.343045  -1.620569
0    118.8
1    118.4
2    118.4
Name: A, dtype: float64


# 数据清洗

In [16]:
import numpy as np
nanlist=np.isnan(IOLdata.PostOPREF)
print(IOLdata[nanlist][0:4])

Empty DataFrame
Columns: [A, K1, K2, L, REF, IOLPower, PostOPREF]
Index: []


In [17]:
IOLdata.PostOPREF[nanlist]=IOLdata.REF[nanlist]
# IOLdata=IOLdata.replace(np.nan,0)
print(IOLdata[nanlist][0:4])
y_data=np.asarray(IOLdata.PostOPREF).T.reshape(-1,1)
x_data=np.asarray(IOLdata)[:,0:6]

print(y_data.shape)
print(x_data.shape)

Empty DataFrame
Columns: [A, K1, K2, L, REF, IOLPower, PostOPREF]
Index: []
(20000, 1)
(20000, 6)


In [18]:
y_mean=np.mean(y_data)
y_std=np.std(y_data)
new_y_data=(y_data-y_mean)/y_std
splitNum=int(new_y_data.shape[0]*0.8)

y_train=new_y_data[0:splitNum]
y_test=new_y_data[splitNum:]


x_mean=np.mean(x_data,axis=0,keepdims=True)
x_std=np.std(x_data,axis=0,keepdims=True)

new_x_data=(x_data-x_mean)/x_std

x_train=new_x_data[:splitNum]
x_test=new_x_data[splitNum:]


# 神经网络简介

# Keras简介

https://keras-cn.readthedocs.io/en/latest/

# 建立深度神经网络

In [19]:
# import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import initializers,optimizers

model = Sequential()
model.add(Dense(units=18,
                input_dim=6,
                activation="relu",
                kernel_initializer=initializers.glorot_normal(seed=0) ))
model.add(Dense(units=12,
                activation="relu",
                kernel_initializer=initializers.glorot_normal()))
model.add(Dense(units=1,
                activation="tanh",
                kernel_initializer=initializers.glorot_normal()))

from keras.optimizers import SGD
model.compile(loss='mean_squared_error', 
              optimizer='rmsprop',
              #optimizer=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08),
              #metrics=['loss']
             )



In [20]:
model.fit(x_train, y_train)#, epochs=200, batch_size=50)

Epoch 1/10
16000/16000 [==============================] - 1s - loss: 0.2715     
Epoch 2/10
16000/16000 [==============================] - 1s - loss: 0.1705     
Epoch 3/10
16000/16000 [==============================] - 0s - loss: 0.1633     
Epoch 4/10
16000/16000 [==============================] - 1s - loss: 0.1571     
Epoch 5/10
16000/16000 [==============================] - 1s - loss: 0.1507     
Epoch 6/10
16000/16000 [==============================] - 0s - loss: 0.1442     
Epoch 7/10
16000/16000 [==============================] - 0s - loss: 0.1396     
Epoch 8/10
16000/16000 [==============================] - 1s - loss: 0.1369     
Epoch 9/10
16000/16000 [==============================] - 0s - loss: 0.1352     
Epoch 10/10
16000/16000 [==============================] - 0s - loss: 0.1340     


结果也太渣了

# 训练拟合

# 使用深度神经网络进行预测